In [153]:
import pathlib
import re
import pandas as pd

In [154]:
%%bash
# ls ../results/spikes3d/drop-ball-noremesh-nref

In [155]:
results = [[
    "../results/ball-wall/3D-noremesh-nref0/2023_03_20_16_16_01_759",
    "../results/ball-wall/3D-noremesh-nref1/2023_03_20_16_29_09_880",
    "../results/ball-wall/3D-noremesh-nref2/2023_03_20_16_38_16_660",
    "../results/ball-wall/3D-noremesh-nref3/2023_03_20_17_26_45_789",
    "../results/ball-wall/3D/2023_03_20_16_01_52_780",
], [
    "../results/masticator/3D-noremesh-nref0/2023_03_20_17_30_56_803",
    "../results/masticator/3D-noremesh-nref1/2023_03_20_17_47_00_837",
    "../results/masticator/3D-noremesh-nref2/2023_03_20_17_56_02_753",
    "../results/masticator/3D-noremesh-nref3/2023_03_20_18_13_10_779",
    "../results/masticator/3D/2023_03_20_17_28_45_698",
], [
#     "../results/rollers/monkey-soft-hard-noremesh-nref0/2023_03_20_19_16_38_739",
#     "../results/rollers/monkey-soft-hard-noremesh-nref1/2023_03_20_19_17_38_820",
#     "../results/rollers/monkey-soft-hard-noremesh-nref2/2023_03_20_19_18_39_023",
#     "../results/rollers/monkey-soft-hard-noremesh-nref3/2023_03_20_19_19_40_403",
#     "../results/rollers/monkey-soft-hard/2023_03_20_19_15_37_662",
# ], [
    "../results/spikes3d/drop-ball-noremesh-nref0/2023_03_20_18_19_18_337",
    "../results/spikes3d/drop-ball-noremesh-nref1/2023_03_20_18_20_12_971",
    "../results/spikes3d/drop-ball-noremesh-nref2/2023_03_20_18_27_18_623",
    "../results/spikes3d/drop-ball-noremesh-nref3/2023_03_20_18_31_22_017",
    "../results/spikes3d/drop-ball/2023_03_20_18_19_13_406",
], [
    "../results/twisting-beam/twisting-beam-noremesh-nref0/2023_03_20_18_18_15_509",
    "../results/twisting-beam/twisting-beam-noremesh-nref1/2023_03_20_18_19_12_461",
    "../results/twisting-beam/twisting-beam-noremesh-nref2/2023_03_20_18_19_12_307",
    "../results/twisting-beam/twisting-beam-noremesh-nref3/2023_03_20_18_19_12_379",
    "../results/twisting-beam/twisting-beam/2023_03_20_18_18_14_155",
]]

In [156]:
for scene in results:
    for dir in scene:
        dir = pathlib.Path(dir)
        stats = {}

        important_lines = []
        try:
            with open(dir / "log.txt") as f:
                lines = f.readlines()
        except:
            print(f"Failed to read {(dir / 'log.txt').resolve()}")
            continue
        for i, line in enumerate(lines):
            if re.match(".*/*  t=.*", line):
                step = int(re.findall(".* (\d+)/.*", line)[0])
                stats[step] = {
                    "time": float(re.findall(".* t=(.+)", line)[0]),
                    "forward": float(re.findall(".*: (\d+\.?\d*) s, .* s, .* s", lines[i+1])[0]),
                    "remeshing": float(re.findall(".*: (\d+\.?\d*) s, .* s, .* s", lines[i+2])[0]),
                    "global_relaxation": float(re.findall(".*: (\d+\.?\d*) s, .* s, .* s", lines[i+3])[0]),
                    "peak_mem": float(re.findall(".*: (\d+\.?\d*) GiB", lines[i+4])[0]),
                }

        df = pd.DataFrame(stats).T
        df.to_csv(dir / "stats.csv")

In [163]:
scenes = ["ball-wall", "masticator", "drop-ball", "twisting-beam"]
nrefs  = ["nref0", "nref1", "nref2", "nref3", "ours"]
df = pd.DataFrame(index=scenes, columns=nrefs)
mem_df = pd.DataFrame(index=scenes, columns=nrefs)
progress = pd.DataFrame(index=scenes, columns=nrefs)
for i, scene in enumerate(results):
    for j, dir in enumerate(scene):
        dir = pathlib.Path(dir)
        stats = pd.read_csv(dir / "stats.csv")
        try:
            df.iloc[i, j] = (stats["forward"] + stats["remeshing"] + stats["global_relaxation"]).mean()
            mem_df.iloc[i, j] = stats["peak_mem"].max()
            progress.iloc[i, j] = stats["time"].max()
        except:
            pass
display(df)
display(mem_df)
display(progress)

,nref0,nref1,nref2,nref3,ours
ball-wall,0.178211,2.063391,21.157302,2563.964856,775.638105
masticator,1.06937,14.904887,213.35809,3030.031415,2387.737547
drop-ball,6.885555,135.303548,3320.449985,NaN,2845.207274
twisting-beam,0.253551,1.585425,67.4121,712.827728,735.636001


,nref0,nref1,nref2,nref3,ours
ball-wall,0.176826,1.134106,8.033726,70.76999,2.800396
masticator,0.967777,0.679218,2.679916,16.437183,2.822708
drop-ball,0.694408,3.869408,27.189373,NaN,1.990879
twisting-beam,0.061943,0.438045,3.072712,21.220558,3.341419


,nref0,nref1,nref2,nref3,ours
ball-wall,0.04,0.04,0.04,0.00132,0.00494
masticator,10.0,10.0,10.0,2.6,3.75
drop-ball,5.0,5.0,0.54,NaN,0.63
twisting-beam,3.0,3.0,3.0,2.5,2.47
